In [1]:
from analytics import operation_builder
from analytics.parser import *
from analytics.visualization import *
import os
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import sent2vec

In [2]:
def select(data):
    columns = data.columns
    valid = []
    for column in columns:
        percentage = len(data.loc[data[column]==1])*1.0/len(data)
        valid.append(percentage)
    data.loc['valid'] = valid
    return data
    

### import data 

In [3]:

list_of_elem_ops_per_pad = dict()
elemOpsCounter = 0
root_of_dbs = "../../data/"


for (dirpath, dirnames, filenames) in os.walk(root_of_dbs):
    for filename in filenames:
        
        if '.sql' in filename:
            path_to_db = os.path.join(dirpath, filename)
            # Fetching the new operations
            list_of_elem_ops_per_main = get_elem_ops_per_pad_from_db(
                path_to_db=path_to_db,
                 editor='sql_dump'
            )# we got the ElementOperations of each pad at beginining

            for pad_name, pad_vals in list_of_elem_ops_per_main.items():
                list_of_elem_ops_per_pad[pad_name + filename[-7:-4]] = pad_vals
        
pads, _, elem_ops_treated = operation_builder.build_operations_from_elem_ops(list_of_elem_ops_per_pad,##  pads with all the operations
                                                                             config.maximum_time_between_elem_ops)

### select best time interval 
must run pad.create_paragraphs_from_ops before since it will set pad start time otherwise no windowOperation can be created 

In [5]:
categrary = {}
#time_interval_list = [20000*i for i in range(4)]
time_interval_list = [180000]
for pad_name in pads.keys():
    pad = pads[pad_name]
    pad.create_paragraphs_from_ops(elem_ops_treated[pad_name]) # create paragraphs and will set the pad start time, must run this before create windowOperation
    label = pad.selectTimeInterval(time_interval_list,10,5)
    categrary[pad_name] = label

In [11]:
import pandas as pd
Columns = [str(x) for x in time_interval_list]
data = pd.DataFrame.from_dict(categrary,orient='index',columns=Columns)
#colum_list = [str(20*i) for i in range(4)]
colum_list = [str(180000)]
data.columns = colum_list
data.head()

,180000
edusearch2014_report-121-15-1MDP,0
edusearch2014_report-121-25-3MDP,1
edusearch2014_report-123-15-1MDP,0
edusearch2014_report-123-25-3MDP,1
edusearch2014_report-126-15-1MDP,0


In [7]:
data.describe()

,180000
count,1071.000000
mean,0.169935
std,0.375751
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [8]:
Des_data = data.describe()
mean_data =Des_data.filter(like='mean',axis=0)

In [9]:
ydata = list(Des_data.loc['mean']*100)
xdata = [20*i for i in range(3,20)]

In [ ]:
Des_data.loc['mean'].plot

### semantic analysis 

In [ ]:
model = sent2vec.Sent2vecModel()
model_name = '../pre_modle/sent2vec/wiki_unigrams.bin'
model.load_model(model_name)  ## import the sent2vec pretrained model

In [ ]:
pad_slope = {}
for pad_name in pads.keys():
    pad = pads[pad_name]
    pad.create_paragraphs_from_ops(elem_ops_treated[pad_name])
    pad.BuildWindowOperation(180000) ## build windowOperation with time interval 180s 
    pad.getTextByWin(True,model) ## get each windowOperations text
    pad.computeDistance() ## compute the distance inside one WindowOperation between different authors
    pad.PlotSimilarityDistribution(False)## this function can plot the similarity distribution in time, and compute the linear fitting slope for the pad
    pad_slope[pad_name] = pad.fitting_slope